In [1]:
from facenet_pytorch import MTCNN, InceptionResnetV1 #pip install mtcnn
import torch
from torchvision import datasets
from torch.utils.data import DataLoader
from PIL import Image
import cv2
import time
import os

ModuleNotFoundError: No module named 'facenet_pytorch'

In [ ]:
#initializing networks
#All code taken from https://www.youtube.com/watch?v=IT53xBR1A7M tutorial

mtcnn0 = MTCNN(image_size=240, margin=0, keep_all=False, min_face_size=40) #This is to recognize faces in the data folders
mtcnn = MTCNN(image_size=240, margin=0, keep_all=True, min_face_size=40) #This is to recognize faces in live camera
resnet = InceptionResnetV1(pretrained='vggface2').eval()

In [ ]:
dataset = datasets.ImageFolder('photos') #photos folder path -> todo create photos folder and add facial recognition
idx_to_class = {i:c for c,i in dataset.class_to_idx.items()} #accessing names of peoples from folder names

def collate_fn(x):
    return x[0]

loader = DataLoader(dataset, collate_fn=collate_fn)

face_list = []
name_list = []
embedding_list = []

#needs to normalize all the pictures to increase accuracy of model
#Need to invesitigate the pretrained model to see if it can recognise faces that are rotated or scaled
for img, idx in loader:
    face, prob = mtcnn0(img, return_prob=True)
    if face is not None and prob>0.92:
        emb = resnet(face.unsqueeze(0))
        embedding_list.append(emb.detach())
        name_list.append(idx_to_class[idx])

#savedata
data = [embedding_list, name_list]
torch.save(data, 'data.pt')

In [ ]:
# Using webcam recognize face

# loading data.pt file
load_data = torch.load('data.pt') 
embedding_list = load_data[0] 
name_list = load_data[1] 

cam = cv2.VideoCapture(1) 

while True:
    ret, frame = cam.read()
    if not ret:
        print("fail to grab frame, try again")
        break
        
    img = Image.fromarray(frame)
    img_cropped_list, prob_list = mtcnn(img, return_prob=True) 
    
    if img_cropped_list is not None:
        boxes, _ = mtcnn.detect(img)
                
        for i, prob in enumerate(prob_list):
            if prob>0.90:
                emb = resnet(img_cropped_list[i].unsqueeze(0)).detach() 
                
                dist_list = [] # list of matched distances, minimum distance is used to identify the person
                
                for idx, emb_db in enumerate(embedding_list):
                    dist = torch.dist(emb, emb_db).item()
                    dist_list.append(dist)

                min_dist = min(dist_list) # get minumum dist value
                min_dist_idx = dist_list.index(min_dist) # get minumum dist index
                name = name_list[min_dist_idx] # get name corrosponding to minimum dist
                
                box = boxes[i] 
                
                original_frame = frame.copy() # storing copy of frame before drawing on it
                

                
                if min_dist<0.90:
                    prob = round(min_dist, 2)
                    frame = cv2.putText(frame, name+' '+str(prob), (int(box[0]), int(box[1])-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0),1, cv2.LINE_AA)
                
                startpoint = (int(box[0]), int(box[1]))
                endpoint = (int(box[2]), int(box[3]))
                frame = cv2.rectangle(frame, startpoint , endpoint, (255,0,0), 2)

    cv2.imshow("IMG", frame)
        
    
    k = cv2.waitKey(1)
    if k%256==27: # ESC
        print('Esc pressed, closing...')
        break #change this break bs
        
    elif k%256==32: # space to save image
        print('Enter your name :')
        name = input()
        
        # create directory if not exists
        if not os.path.exists('photos/'+name):
            os.mkdir('photos/'+name)
            
        img_name = "photos/{}/{}.jpg".format(name, int(time.time()))
        cv2.imwrite(img_name, original_frame)
        print(" saved: {}".format(img_name))
        
        
cam.release()
cv2.destroyAllWindows()

Esc pressed, closing...


In [ ]:
cam.release()
cv2.destroyAllWindows()